https://www.cs.toronto.edu/~kriz/cifar.html

In [29]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [44]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# show image
img = Image.fromarray(x_train[1], 'RGB')
img.show()

plt.imshow(np.dot(x_train[1], [0.2989, 0.5870, 0.1140]), cmap=plt.get_cmap('gray'))
plt.show()

In [41]:
x_train, x_test = np.dot(x_train[...,:3], [0.2989, 0.5870, 0.1140]), np.dot(x_test[...,:3], [0.2989, 0.5870, 0.1140])
x_train, x_test = x_train/255 , x_test/255
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.15, shuffle=True)
#y_train = tf.one_hot(y_train.astype(np.int32), depth=10)

In [32]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(32,32)),
        tf.keras.layers.Dense(512, activation="relu", name="layer1"),
        tf.keras.layers.Dense(60, activation="relu", name="layer2"),
        tf.keras.layers.Dense(10, activation="softmax", name="output_layer"),
    ]
)

In [33]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [34]:
EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
ModelCheckpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoint", save_best_only=True)
TensorBoard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
model_history = model.fit(x=x_train, y=y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[EarlyStopping_callback, ModelCheckpoint_callback, TensorBoard_callback])

Epoch 1/50
1329/1329 [==============================] - 6s 4ms/step - loss: 2.1422 - accuracy: 0.1999 - val_loss: 2.0985 - val_accuracy: 0.2424
Epoch 2/50
1329/1329 [==============================] - 6s 5ms/step - loss: 1.9536 - accuracy: 0.2944 - val_loss: 1.9706 - val_accuracy: 0.2829
Epoch 3/50
1329/1329 [==============================] - 6s 4ms/step - loss: 1.8762 - accuracy: 0.3262 - val_loss: 1.9358 - val_accuracy: 0.3119
Epoch 4/50
1329/1329 [==============================] - 6s 4ms/step - loss: 1.8276 - accuracy: 0.3472 - val_loss: 1.8991 - val_accuracy: 0.3188
Epoch 5/50
1329/1329 [==============================] - 6s 4ms/step - loss: 1.7900 - accuracy: 0.3615 - val_loss: 1.8658 - val_accuracy: 0.3376
Epoch 6/50
1329/1329 [==============================] - 6s 4ms/step - loss: 1.7601 - accuracy: 0.3742 - val_loss: 1.8026 - val_accuracy: 0.3524
Epoch 7/50
1329/1329 [==============================] - 6s 4ms/step - loss: 1.7301 - accuracy: 0.3857 - val_loss: 1.8279 - val_accuracy:

In [40]:
%load_ext tensorboard
%tensorboard --logdir=./logs --port=8877

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
model.evaluate(x=x_test, y=y_test, verbose=1)

(10000, 32, 32)

In [ ]:
pred = model.predict(x_test)

In [ ]:
def plotmodelhistory(history): 
    fig, axs = plt.subplots(1,2,figsize=(15,5)) 
    # summarize history for accuracy
    axs[0].plot(history.history['accuracy']) 
    axs[0].plot(history.history['val_accuracy']) 
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy') 
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['train', 'validate'], loc='upper left')
    # summarize history for loss
    axs[1].plot(history.history['loss']) 
    axs[1].plot(history.history['val_loss']) 
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss') 
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['train', 'validate'], loc='upper left')
    plt.show()

plotmodelhistory(model_history)

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None, cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.
    """
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    
    ax.set_xlabel('Predicted Label') 
    ax.set_ylabel('True Label')
    
    return im, cbar

def annotate_heatmap(im, data=None, fmt="d", threshold=None):
    """
    A function to annotate a heatmap.
    """
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            text = im.axes.text(j, i, format(data[i, j], fmt), horizontalalignment="center",
                                 color="white" if data[i, j] > thresh else "black")
            texts.append(text)

    return texts
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(pred, axis=1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = pred[errors]
Y_true_errors = Y_true[errors]
X_test_errors = x_test[errors]

cm = confusion_matrix(Y_true, Y_pred_classes) 
thresh = cm.max() / 2.

fig, ax = plt.subplots(figsize=(12,12))
im, cbar = heatmap(cm, labels, labels, ax=ax,
                   cmap=plt.cm.Blues, cbarlabel="count of predictions")
texts = annotate_heatmap(im, data=cm, threshold=thresh)

fig.tight_layout()
plt.show()

In [ ]:
print(classification_report(Y_true, Y_pred_classes))

In [ ]:
R = 3
C = 5
fig, axes = plt.subplots(R, C, figsize=(12,8))
axes = axes.ravel()

misclassified_idx = np.where(Y_pred_classes != Y_true)[0]
for i in np.arange(0, R*C):
    axes[i].imshow(x_test[misclassified_idx[i]])
    axes[i].set_title("True: %s \nPredicted: %s" % (labels[Y_true[misclassified_idx[i]]], 
                                                  labels[Y_pred_classes[misclassified_idx[i]]]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

In [ ]:
model.save("cifar10_.h5")
tf.keras.models.load_model("cifar10_.h5")